In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mat = np.load('DtoP_transition_matrix.npy')

In [ ]:
# Parameters (example values — adjust as needed)
params = {
    # birth
    'beta': {'F': mat[4,1], 'H': mat[4,2], 'S': mat[4,3], 'M': mat[4,4]},
    # death
    'delta': {'F': mat[3,0], 'H': mat[2,0], 'S': mat[1,0], 'M': mat[0,0]},
    # transition
    'gamma': {
        # from F to X
        'F': {'H': mat[3,2], 'S': mat[3,3], 'M': mat[3,4]},
        # from H to X
        'H': {'F': mat[2,1], 'S': mat[2,3], 'M': mat[2,4]},
        # from S to X
        'S': {'F': mat[1,1], 'H': mat[1,2], 'M': mat[1,4]},
        # from M to X
        'M': {'F': mat[0,1], 'H': mat[0,2], 'S': mat[0,3]}
    }
}

# State initialization: [F, H, S, M]
state = np.array([2, 1, 0, 0], dtype=int)
time = 0.0
Tmax = 100
states_over_time = [(time, *state)]

def get_reactions(state):
    reactions = []
    labels = []

    # Births (independent of current state)
    for i, X in enumerate(['F', 'H', 'S', 'M']):
        reactions.append((lambda s, i=i: 1, np.eye(4, dtype=int)[i]))
        labels.append(f"birth_{X}")

    # Deaths (proportional to current count)
    for i, X in enumerate(['F', 'H', 'S', 'M']):
        reactions.append((lambda s, i=i: s[i], -np.eye(4, dtype=int)[i]))
        labels.append(f"death_{X}")

    # Transitions (proportional to donor population)
    for i, from_X in enumerate(['F', 'H', 'S', 'M']):
        for j, to_X in enumerate(['F', 'H', 'S', 'M']):
            if i != j:
                rate = params['gamma'][from_X][to_X]
                update = np.zeros(4, dtype=int)
                update[i] -= 1
                update[j] += 1
                reactions.append((lambda s, i=i, r=rate: r * s[i], update))
                labels.append(f"{from_X}->{to_X}")

    return reactions, labels

# Simulation loop
reactions, labels = get_reactions(state)
while time < Tmax:
    rates = np.array([r(state) * (params['beta'].get(X, 1) if 'birth' in lbl else 
                                  params['delta'].get(X, 1) if 'death' in lbl else 1)
                      for r, lbl, X in zip(reactions, labels, ['F','H','S','M']*5)])
    
    total_rate = np.sum(rates)
    if total_rate == 0:
        break
    
    # Time to next reaction
    dt = np.random.exponential(1 / total_rate)
    time += dt

    # Choose reaction
    reaction_index = np.random.choice(len(rates), p=rates / total_rate)
    state += reactions[reaction_index][1]

    # Save state
    states_over_time.append((time, *state))

# Plotting
times, F, H, S, M = zip(*states_over_time)
plt.figure(figsize=(10, 6))
plt.plot(times, F, label='F')
plt.plot(times, H, label='H')
plt.plot(times, S, label='S')
plt.plot(times, M, label='M')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Gillespie Simulation of 4-State System')
plt.legend()
plt.grid(True)
plt.show()

TypeError: 'tuple' object is not callable